[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/UMCF.ipynb)

# ユーザー間型メモリベース法協調フィルタリング

In [20]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
import numpy as np
# ピアソンの相関係数
def peason_coefficient(u: np.ndarray, v: np.ndarray) -> float:
    u_diff = u - np.mean(u)
    v_diff = v - np.mean(v)
    numerator = np.dot(u_diff, v_diff)
    denominator = np.sqrt(sum(u_diff ** 2)) * np.sqrt(sum(v_diff ** 2))
    if denominator == 0:
        return 0.0
    return numerator / denominator


In [6]:
from collections import defaultdict

# 評価値をユーザー×映画の行列に変換
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))

# 予測対象のユーザーと映画の組
movie_rating_predict = movielens_test.copy()
pred_user2items = defaultdict(list)

In [8]:
# 愚直に類似度を計算（こちらの計算はとても重いです。そのため、途中でbreakを入れています。速度向上のために、ライブラリを使った実装も次のセルで紹介します。）

# 予測対象のユーザーID
test_users = movie_rating_predict.user_id.unique()

# 予測対象のユーザー(ユーザー1）に注目する
for user1_id in test_users:
    similar_users = []
    similarities = []
    avgs = []

    # ユーザ−１と評価値行列中のその他のユーザー（ユーザー２）との類似度を算出する
    for user2_id in user_movie_matrix.index:
        if user1_id == user2_id:
            continue

        # ユーザー１とユーザー２の評価値ベクトル
        u_1 = user_movie_matrix.loc[user1_id, :].to_numpy()
        u_2 = user_movie_matrix.loc[user2_id, :].to_numpy()

        # `u_1` と `u_2` から、ともに欠損値でない要素のみ抜き出したベクトルを取得
        common_items = ~np.isnan(u_1) & ~np.isnan(u_2)

        # 共通して評価したアイテムがない場合はスキップ
        if not common_items.any():
            continue

        u_1, u_2 = u_1[common_items], u_2[common_items]

        # ピアソンの相関係数を使ってユーザー１とユーザー２の類似度を算出
        rho_12 = peason_coefficient(u_1, u_2)

        # ユーザー1との類似度が0より大きい場合、ユーザー2を類似ユーザーとみなす
        if rho_12 > 0:
            similar_users.append(user2_id)
            similarities.append(rho_12)
            avgs.append(np.mean(u_2))

    # ユーザー１の平均評価値
    avg_1 = np.mean(user_movie_matrix.loc[user1_id, :].dropna().to_numpy())

    # 予測対象の映画のID
    test_movies = movie_rating_predict[movie_rating_predict["user_id"] == user1_id].movie_id.values
    # 予測できない映画への評価値はユーザー１の平均評価値とする
    movie_rating_predict.loc[(movie_rating_predict["user_id"] == user1_id), "rating_pred"] = avg_1

    if similar_users:
        for movie_id in test_movies:
            if movie_id in movie_id2index:
                r_xy = user_movie_matrix.loc[similar_users, movie_id].to_numpy()
                rating_exists = ~np.isnan(r_xy)

                # 類似ユーザーが対象となる映画への評価値を持っていない場合はスキップ
                if not rating_exists.any():
                    continue

                r_xy = r_xy[rating_exists]
                rho_1x = np.array(similarities)[rating_exists]
                avg_x = np.array(avgs)[rating_exists]
                r_hat_1y = avg_1 + np.dot(rho_1x, (r_xy - avg_x)) / rho_1x.sum()

                # 予測評価値を格納
                movie_rating_predict.loc[
                    (movie_rating_predict["user_id"] == user1_id)
                    & (movie_rating_predict["movie_id"] == movie_id),
                    "rating_pred",
                ] = r_hat_1y
    break # 計算が重いため、forループの一回目で終わるようにしています。各変数にどんな値が入っているかを確認してもらえれば、アルゴリズムの理解が深まります

In [ ]:
!pip install surprise

In [14]:
# Surpriseを利用した実装

from surprise import KNNWithMeans, Reader
from surprise import Dataset as SurpriseDataset
# Surprise用にデータを加工
reader = Reader(rating_scale=(0.5, 5))
data_train = SurpriseDataset.load_from_df(
    movielens_train[["user_id", "movie_id", "rating"]], reader
).build_full_trainset()

sim_options = {"name": "pearson", "user_based": True}  # 類似度を計算する方法を指定する  # False にするとアイテムベースとなる
knn = KNNWithMeans(k=30, min_k=1, sim_options=sim_options)
knn.fit(data_train)

# 学習データセットで評価値のないユーザーとアイテムの組み合わせを準備
data_test = data_train.build_anti_testset(None)
predictions = knn.test(data_test)

def get_top_n(predictions, n=10):
    # 各ユーザーごとに、予測されたアイテムを格納する
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # ユーザーごとに、アイテムを予測評価値順に並べ上位n個を格納する
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [d[0] for d in user_ratings[:n]]

    return top_n

pred_user2items = get_top_n(predictions, n=10)

average_score = movielens_train.rating.mean()
pred_results = []
for _, row in movielens_test.iterrows():
    user_id = row["user_id"]
    movie_id = row["movie_id"]
    # 学習データに存在せずテストデータにしか存在しないユーザーや映画についての予測評価値は、全体の平均評価値とする
    if user_id not in user_id2index or movie_id not in movie_id2index:
        pred_results.append(average_score)
        continue
    pred_score = knn.predict(uid=user_id, iid=movie_id).est
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

pred_user2items

Computing the pearson similarity matrix...
Done computing similarity matrix.


defaultdict(list,
            {139: [6319, 2721, 2131, 3415, 1121, 2509, 4221, 4418, 4509, 5101],
             149: [2494,
              26425,
              137,
              2930,
              1477,
              3473,
              27255,
              2911,
              2425,
              31116],
             182: [6319, 3415, 3473, 146, 1121, 2509, 4221, 4418, 4509, 6122],
             215: [1564, 3365, 1204, 1131, 2931, 2938, 1192, 1489, 2679, 1147],
             281: [137,
              1477,
              26425,
              6319,
              27255,
              2911,
              4850,
              5069,
              2494,
              31116],
             326: [1192, 2679, 3341, 7064, 32316, 3415, 3473, 4208, 4453, 146],
             351: [3473,
              6122,
              2911,
              137,
              27255,
              3777,
              1477,
              2930,
              26425,
              31116],
             357: [7064,
              

In [16]:
# user_id=2のユーザーが学習データで評価を付けた映画一覧
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [17]:
pred_user2items[2]

[3473, 4135, 4850, 2425, 2704, 137, 27255, 26425, 8712, 31116]

In [18]:
# user_id=2に対するおすすめ(1198, 1196, 1097)
movies[movies.movie_id.isin([3473, 4135, 4850])]

,movie_id,title,genre,tag
3386,3473,Jonah Who Will Be 25 in the Year 2000 (Jonas q...,[Comedy],"[cerebral, humorous, intersecting lives, intim..."
4043,4135,"Monster Squad, The (1987)","[Adventure, Comedy, Horror]",[can't remember]
4756,4850,Spriggan (Supurigan) (1998),"[Action, Animation, Sci-Fi]","[library, anime, spectacle, over expositive]"
